In [103]:
import pandas as pd
import seaborn as sns

In [104]:
data = pd.read_csv('data/health.csv')
data = data.drop(columns = ['Unnamed: 0', 'index', 'CASEID'])

### Create Suicide Risk

- plan1: think(1), plan(2), attempt(3) : doesnt work out, attempt doesn't ensure plan, and plan doesnt ensure think: for people who actually tried, half of them didn't have any plan, and some of them havent thought ab killing themselves: something we could put into the slide as an interesting finding (might include some graphs as well)

- plan 2: think(1), think+plan(2), think+plan+attempt(3), but what ab those who are attemp only, or plan only, or attempt+think?

- plan 3: think (+1), plan (+2), attempt (+3)


In [105]:
suic = data[['suic_thought_pst_12month', 'suic_thought_pst_yr', 'suic_plan', 'suic_attempt']]

In [106]:
suicide_risk = suic['suic_thought_pst_yr'] + 2*suic['suic_plan'] + 3*suic['suic_attempt']
suicide_risk.value_counts()

0.0    262867
1.0     11561
3.0      2781
2.0      1369
6.0      1312
4.0       420
5.0        15
dtype: int64

In [107]:
data = data.assign(suicide_risk = suicide_risk)
data = data.drop(columns = ['YEAR', 'Age_First_Use', 'Year_First_Use', 
                            'CPN_Meth_Flag', 'Any_Drug_Past_Year', 'Any_Drug_Past_Month'])


In [108]:
data.columns

Index(['SNRLGSVC', 'SNRLGIMP', 'SNRLDCSN', 'SNRLFRND', 'Tabacco_Flag',
       'Alcohol_Flag', 'Marijunana_Flag', 'Marijuana_Only_Flag',
       'Cocaine_Flag', 'Crack_Flag', 'Herion_Flag', 'Hallcinogens_Flag',
       'Inhalants_Flag', 'Painkiller_Flag', 'Tranquilizer_Flag',
       'Stimulant Flag', 'Meth_Flag', 'Sedatives Flag', 'Any_Drug_Flag',
       'Drug_Not_Marijunana', 'Ever_Steal_50', 'Sell_Illegal_Drugs',
       'Ever_Booked_Arrested', 'Drug_or_Alc_Past_year', 'inpatient',
       'outpatient', 'needed_not_rcvd', 'rcvd_any_tmt',
       'suic_thought_pst_12month', 'psych_distress_month', 'psych_distress_yr',
       'suic_thought_pst_yr', 'suic_plan', 'suic_attempt', 'whodas_score',
       'predicted_ment_ill', 'svrl_days_depressed', 'lifetime_mde',
       'pst_yr_mde', 'CATAG6', 'HEALTH', 'IRSEX', 'IRMARIT', 'EDUCCAT2',
       'NEWRACE2', 'EMPSTATY', 'suicide_risk'],
      dtype='object')

In [109]:
data = data[data['SNRLGSVC'] != '99']
data = data[data['SNRLGIMP'] != '99']
data = data[data['SNRLDCSN'] != '99']
data = data[data['SNRLFRND'] != '99']
data = data[data['Sell_Illegal_Drugs'] != '97']
data = data[data['Sell_Illegal_Drugs'] != '94']

In [110]:
data['SNRLGSVC'].replace(['0', '1-2', '3-5', '6-24', '25-52', '52+', '99'],
                        [0, 1, 2, 3, 4, 5, 6], inplace=True)
data['SNRLGIMP'].replace(['Strongly Agree', 'Agree', 'Disagree', 'Strongly Disagree'],
                        [0, 1, 2, 3], inplace=True)
data['SNRLDCSN'].replace(['Strongly Agree', 'Agree', 'Disagree', 'Strongly Disagree'],
                        [0, 1, 2, 3], inplace=True)
data['SNRLFRND'].replace(['Strongly Agree', 'Agree', 'Disagree', 'Strongly Disagree'],
                        [0, 1, 2, 3], inplace=True)
data['Ever_Steal_50'].replace(['0', '1-2', '3-5', '6-9', '>=10'],
                        [0, 1, 2, 3, 4], inplace=True)
data['Sell_Illegal_Drugs'].replace(['0', '1-2', '3-5', '6-9', '>=10'],
                        [0, 1, 2, 3, 4], inplace=True)
data['Ever_Booked_Arrested'].replace(['No', 'Yes'],
                        [0, 1], inplace=True)
data['Drug_or_Alc_Past_year'].replace(['No/Unknown', 'Yes'],
                        [0, 1], inplace=True)

data['CATAG6'].replace(['A-18-25', 'A-26-34', 'A-35-49','A-50-64','A-65' ],
                        [0, 1, 2, 3, 4], inplace=True)
data['HEALTH'].replace(['H_Very_good', 'H_Excellent', 'H_Good', 'H_Fair', 'H_Poor'],
                        [0, 1, 2, 3, 4], inplace=True)
data['IRSEX'].replace(['Male', 'Female'],
                        [0, 1], inplace=True)
data['IRMARIT'].replace(['Not_Married', 'Married', 'Divorced_Separated', 'Widowed'],
                        [0, 1, 2, 3], inplace=True)
data['EDUCCAT2'].replace(['Less_than_high_school', 'High_school_graduate', 'Some_college','College_graduate'],
                        [0, 1, 2, 3], inplace=True)
data['NEWRACE2'].replace(['Black/Afr_Am', 'White', 'Hispanic', 'more_than_one_race', 'Asian',
 'Native_Am/AK_Native', 'Native_HI/Other_Pac_Isl'],
                        [0, 1, 2, 3, 4, 5, 6], inplace=True)
data['EMPSTATY'].replace(['E_full_time', 'E_Other', 'E_Unemployed', 'E_part_time'],
                        [0, 1, 2, 3], inplace=True)

In [111]:
map_col = {'SNRLGSVC':'RELIG_SERVICES_12month', 'SNRLGIMP':'RELIG_IMPORTANT', 
           'SNRLDCSN':'RELIG_influ_decision', 'SNRLFRND': 'friend_SHARE_REL', 
           'CATAG6':'age'}
data = data.rename(columns=map_col)

In [121]:
import numpy as np
def normalize(series):
    """
    normalize a series
    """
    array = np.array(series)
    array_norm = (array-np.min(array))/(np.max(array)-np.min(array))
    return pd.Series(array_norm)

norm_data = pd.DataFrame()
for col in data.columns:
    norm_data[col] = normalize(data[col])

In [123]:
norm_data.to_csv('data/new_health.csv')

In [90]:
# from sklearn.manifold import TSNE, MDS

# X = data[['Tabacco_Flag', 'Alcohol_Flag', 'Marijunana_Flag', 'Marijuana_Only_Flag', 'Cocaine_Flag']].values.tolist()
# y = pd.factorize(data['suicide_risk'])[0]
# y = y.tolist()

# dims = TSNE(random_state=42).fit_transform(X)
# dim1, dim2 = dims[:, 0], dims[:, 1]
# sns.scatterplot(x=dim1, y=dim2, hue=t, palette='tab10', legend=False)
# plt.title('t-SNE')

In [5]:
suic[suic['suic_thought_pst_yr'] == 1.0].suic_plan.value_counts()
#for those who have thought about it, few of them made plan

suic[suic['suic_thought_pst_yr'] == 0.0].suic_plan.value_counts()
#for those who have never thought about it, small porportion of them made plan

suic[suic['suic_thought_pst_yr'] == 1.0].suic_attempt.value_counts()
#for those who have thought ab it, only have few actually tried

suic[suic['suic_thought_pst_yr'] == 0.0].suic_attempt.value_counts()
#for those who have never thought ab it, a very small porportion of them have few actually tried

suic[suic['suic_attempt'] == 1.0].suic_plan.value_counts()
#for those who have attempted suicide, half made plan

suic[suic['suic_attempt'] == 1.0].suic_thought_pst_yr.value_counts()
#for those who have attempted suicide, majority thought about it

suic[suic['suic_plan'] == 1.0].suic_thought_pst_yr.value_counts()
#for those who have made plan, majority have thought ab it

suic[suic['suic_plan'] == 0.0].suic_thought_pst_yr.value_counts()
#for those who dont have any plan, only a few thought about it

0.0    11981
1.0     3327
Name: suic_plan, dtype: int64